# 第8章 评分预测问题

&emsp;&emsp;评分预测问题最基本的数据集就是用户评分数据集，评分预测问题就是如何通过已知的用户历史评分记录预测未知的用户评分记录。

## 离线实验方法

&emsp;&emsp;在给定用户评分数据集后，将数据集按照一定的方式分成训练集和测试集，然后根据测试集建立用户兴趣模型来预测测试集中的用户评分。对于测试集中的一对用户和物品$(u,i)$，用户$u$对物品$i$的真实评分是$r_{ui}$，而推荐算法预测的用户$u$对物品$i$的评分为$\hat{r}_{ui}$，那么一般可以用均方根误差RMSE度量预测的精度：$$\text{RMSE}=\frac{\sqrt{\displaystyle \sum_{(u,i) \in T}(r_{ui} - \hat{r}_{ui})^2}}{|\text{Test}|}$$

## 评分预测算法

### 平均值

1. 全局平均值  
训练集中所有评分记录的评分平均值：$$\hat{r}_{ui}  = \mu=\frac{\displaystyle \sum_{(u,i) \in Train }r_{ui}}{\displaystyle \sum_{(u,i) \in Train} 1}$$

2. 用户评分平均值  
用户$u$的评分平均值$\bar{r}_u$定义为用户$u$在训练集中所有评分的平均值：$$\hat{r}_{ui}=\bar{r}_u=\frac{\displaystyle \sum_{i \in N(u)}r_{ui}}{\displaystyle \sum_{i \in N(u)} 1} $$

3. 物品评分平均值  
物品$i$的评分平均值$\bar{r}_i$定义为物品$i$在训练集中接受的所有评分的平均值：$$\hat{r}_{ui}=\bar{r}_i=\frac{\displaystyle \sum_{u \in N(i)} r_{ui}}{\displaystyle \sum_{u \in N(i)} 1}$$

4. 用户分类对物品分类的平均值  
&emsp;&emsp;假设有两个分类函数，一个是用户分类函数$\phi$，一个是物品分类函数$\varphi$。 $\phi(u)$定义了用户$u$所属的类， $\varphi(i)$定义了物品$i$所属的类，利用训练集中同类用户对同类物品评分的平均值预测用户对物品的评分：$$\hat{r}_ui=\frac{\displaystyle \sum_{(v,j) \in Train, \phi(u)=\phi(v), \varphi(i)=\varphi(j)} r_{vj}}{\displaystyle \sum_{(v,j) \in Train, \phi(u)=\phi(v), \varphi(i)=\varphi(j)} 1}$$用户评分平均值和物品评分平均值都是类类平均值的一种特例：
 - 如果定义$\phi(u)=0, \varphi(i)=0$，那么$\hat{r}_{ui}$就是全局平均值。
 - 如果定义$\phi(u)=u, \varphi(i)=0$，那么$\hat{r}_{ui}$就是用户评分平均值。
 - 如果定义$\phi(u)=0, \varphi(i)=i$，那么$\hat{r}_{ui}$就是物品评分平均值。
 
除了这3种特殊的平均值，在用户评分数据上还可以定义很多不同的分类函数：
- 用户和物品的平均分  
  &emsp;&emsp;对于一个用户，可以计算他的评分平均分。然后将所有用户按照评分平均分从小到大排序，并将用户按照平均分,平均分成N类。物品也可以用同样的方式分类。
- 用户活跃度和物品流行度   
  &emsp;&emsp;对于一个用户，将他评分的物品数量定义为他的活跃度。得到用户活跃度之后，可以将用户通过活跃度从小到大排序，然后平均分为N类。物品的流行度定义为给物品评分的用户数目，物品也可以按照流行度均匀分成N类。

In [1]:
import os, sys
import pandas as pd
import pandas as pd
import math

from main.chapter8.cluster import Cluster, IdCluster
from main.chapter8.item_cluster import ItemPopularityCluster, ItemVoteCluster
from main.chapter8.user_cluster import UserActivityCluster, UserVoteCluster
from main.util import movielen_reader
from main.chapter8.predict_all import PredictAll

PROJECT_ROOT = os.path.dirname(sys.path[0])

In [2]:
movie_len_path = os.path.join(PROJECT_ROOT, "src/data/ml-1m/ratings.dat")

# 加载数据集
movie_len_dataset = movielen_reader.load_data(movie_len_path)
train_dataset, test_dataset = movielen_reader.split_data(movie_len_dataset)

In [3]:
def RMSE(records):
    """计算RMSE
        @param records: 预测评价与真实评价记录的一个list
        @return: RMSE
    """
    numerator = sum([(r.rate - r.predict) ** 2 for r in records])
    denominator = float(len(records))
    return math.sqrt(numerator / denominator)

In [5]:
UserGroups = [Cluster, IdCluster, Cluster, UserActivityCluster, UserActivityCluster, Cluster, IdCluster,
              UserActivityCluster, UserVoteCluster, UserVoteCluster, Cluster, IdCluster, UserVoteCluster]
ItemGroups = [Cluster, Cluster, IdCluster, Cluster, IdCluster, ItemPopularityCluster, ItemPopularityCluster,
              ItemPopularityCluster, Cluster, IdCluster, ItemVoteCluster, ItemVoteCluster, ItemVoteCluster]

metric = []
for i in range(len(UserGroups)):
    user_group = UserGroups[i]
    item_group = ItemGroups[i]
    # 模型训练
    model = PredictAll(train_dataset, test_dataset, user_group, item_group)
    train_dataset = model.fit()
    test_dataset = model.predict(test_dataset)
    
    # 计算RMSE
    train_metric = RMSE(train_dataset)
    test_metric = RMSE(test_dataset)
    metric.append((user_group.__name__, item_group.__name__, train_metric, test_metric))

In [6]:
pd.DataFrame(
    data=metric,
    index=[i for i in range(len(UserGroups))],
    columns=['UserGroup', 'ItemGroup', 'TrainRMSE', 'TestRMSE']
)

,UserGroup,ItemGroup,TrainRMSE,TestRMSE
0,Cluster,Cluster,1.116067,1.126368
1,IdCluster,Cluster,1.026942,1.047887
2,Cluster,IdCluster,0.973980,0.987342
3,UserActivityCluster,Cluster,1.109116,1.126040
4,UserActivityCluster,IdCluster,0.972588,0.984734
5,Cluster,ItemPopularityCluster,1.089944,1.105140
6,IdCluster,ItemPopularityCluster,0.998388,1.023047
7,UserActivityCluster,ItemPopularityCluster,1.085856,1.104758
8,UserVoteCluster,Cluster,1.035265,1.125829
9,UserVoteCluster,IdCluster,0.919145,0.984236


### 基于邻域的方法

基于用户的邻域算法和基于物品的邻域算法都可以应用到评分预测中。  

1. 基于用户的邻域算法  
&emsp;&emsp;基于用户的邻域算法认为预测一个用户对一个物品的评分，需要参考和这个用户兴趣相似的用户对该物品的评分，即：$$\hat{r}_{ui} = \bar{r}_u + \frac{\displaystyle \sum_{v \in S(u,K) \cap N(i)} w_{uv}\left(r_{vi}-\bar{r}_v\right)}{\displaystyle \sum_{v \in S(u,K) \cap N(i)}\left|w_{uv}\right|}$$&emsp;&emsp;其中，$S(u, K)$是和用户$u$兴趣最相似的$K$个用户的集合， $N(i)$是对物品$i$评过分的用户集合，$r_{vi}$是用户$v$对物品$i$的评分，$\bar{r}_v$是用户$v$对他评过分的所有物品评分的平均值。用户之间的相似度$w_{uv}$可以通过皮尔逊系数计算：$$w_{uv}=\frac{\displaystyle \sum_{i \in I}(r_{ui}-\bar{r}_u)(r_{vi}-\bar{r}_v)}{\sqrt{\displaystyle \sum_{i \in I}(r_{ui}-\bar{r}_u)^2 \sum_{i \in I}(r_{vi}-\bar{r}_v)^2} }$$

2. 基于物品的邻域算法  
&emsp;&emsp;基于物品的邻域算法在预测用户u对物品i的评分时，会参考用户u对和物品i相似的其他物品
的评分，即：$$
\hat{r}_{ui}=\bar{r}_{i}+\frac{\displaystyle \sum_{j \in S(u, K) \cap N(u)} w_{i j}\left(r_{u j}-\bar{r}_{i}\right)}{\displaystyle \sum_{j \in S(i, K) \cap N(u)}\left|w_{i j}\right|}
$$&emsp;&emsp;其中，$S(i, K)$是和$i$最相似的物品集合，$N(u)$是用户$u$评过分的物品集合，$w_{ij}$是物品之间的相似度，$\bar{r}_i$是物品$i$的平均分。  
&emsp;&emsp;至于如何计算物品之间的相似度，有如下三种方式：
 - 余弦相似度$$
w_{i j}=\frac{\displaystyle \sum_{u \in U} r_{u i} \cdot r_{u j}}{\sqrt{\displaystyle \sum_{u \in U} r_{u i}^{2} \sum_{u \in U} r_{u j}^{2}}}
$$
 - 皮尔逊系数$$
w_{i j}=\frac{\displaystyle \sum_{u \in U}\left(r_{u i}-\bar{r}_{i}\right) \cdot\left(r_{u j}-\bar{r}_{j}\right)}{\sqrt{\displaystyle \sum_{u \in U}\left(r_{u i}-\bar{r}_{i}\right)^{2} \sum_{u \in U}\left(r_{u j}-\bar{r}_{j}\right)^{2}}}
$$
 - 被Sarwar称为修正的余弦相似度$$
w_{i j}=\frac{\displaystyle \sum_{u \in U}\left(r_{u i}-\bar{r}_{u}\right) \cdot\left(r_{u j}-\bar{r}_{u}\right)}{\sqrt{\displaystyle \sum_{u \in U}\left(r_{u i}-\bar{r}_{u}\right)^{2} \sum_{u \in U}\left(r_{u j}-\bar{r}_{u}\right)^{2}}}
$$

### 隐语义模型及矩阵分解

&emsp;&emsp;用户的评分行为可以表示成一个评分矩阵$R$，其中$R[u][i]$就是用户$u$对物品$i$的评分。但是，用户不会对所有的物品评分，所以这个矩阵里有很多元素都是空的，这些空的元素称为缺失值（missing value）。因此，评分预测从某种意义上说就是填空，如果一个用户对一个物品没有评过分，那么推荐系统就要预测这个用户是否会对这个物品评分以及会评几分。  

1. 传统的SVD分解  
&emsp;&emsp;给定$m$个用户和$n$个物品，和用户对物品的评分矩阵$R \in \mathbb{R}^{m \times n}$，首先需要对评分矩阵中的缺失值进行简单地补全，比如用全局平均值，或者用户/物品平均值补全，得到补全后的矩阵$R'$。接着，可以用SVD分解将$R'$分解成如下形式：$$R'=U^T S V$$&emsp;&emsp;其中$U \in \mathbb{R}^{k \times m}, V \in \mathbb{R}^{k \times n}$是两个正交矩阵， $S \in \mathbb{R}^{k \times k}$是对角阵，对角线上的每一个元素都是矩阵的奇异值。为了对$R'$进行降维，可以取最大的$f$个奇异值组成对角矩阵$S_f$，并且找到这$f$个奇异值中每个值在$U$、$V$矩阵中对应的行和列，得到$U_f$、$V_f$，从而可以得到一个降维后的评分矩阵：$$R'_f=U_f^T S_f V_f$$&emsp;&emsp;其中，$R'_f(u,i)$就是用户$u$对物品$i$评分的预测值。  
这种早期的方法有如下两个缺点：
 - 该方法首先需要用一个简单的方法补全稀疏评分矩阵，这种空间的需求在实际系统中是不可能接受的。
 - 该方法依赖的SVD分解方法的计算复杂度很高，特别是在稠密的大规模矩阵上更是非常慢。

2. Simon Funk的SVD分解  
该方法称为Latent Factor Model（简称为LFM）。
将评分矩阵$R$分解为两个低维矩阵相乘：$$\hat{R} = P^T Q$$其中，$P \in \mathbb{R}^{f \times m}$和$Q \in \mathbb{R}^{f \times n}$是两个降维后的矩阵。那么，对于用户$u$对物品$i$的评分的预测值$\hat{R}(u,i)=\hat{r}_{ui}$，可以通过如下公式计算：$$\hat{r}_{ui}=\sum_f P_{uf} q_{if}$$其中$p_{uf}=P(u,f),q_{if}=Q(i,f)$。得到这个公式之后，就直接通过训练集中的观察值，利用最小化RMSE学习P、Q矩阵，损失函数如下：$$
C(p, q)=\sum_{(u, i) \in \text { Train }}\left(r_{u i}-\sum_{f=1}^F p_{u f} q_{i f}\right)^2+\lambda\left(\left\|p_u \right\|^2+\left\|q_i\right\|^2 \right)$$
采用经典的SGD（梯度下降法）进行优化，其梯度公式为：$$
\begin{aligned}
&\frac{\partial C}{\partial p_{u f}}=-2 q_{k}+2 \lambda p_{u k}\\
&\frac{\partial C}{\partial p_{i f}}=-2 p_{u k}+2 \lambda q_{i k}
\end{aligned}
$$参数沿着最速下降方向向前推进，得到递推公式：$$
\begin{array}{l}
p_{u f}=p_{u f}+\alpha\left(q_{i k}-\lambda p_{u k}\right) \\
q_{i f}=q_{i f}+\alpha\left(p_{u k}-\lambda q_{i k}\right)
\end{array}
$$

3. 加入偏置项后的LFM  
预测公式：$$\hat{r}_{u i}=\mu+b_u+b_i+p_u^T \cdot q_i$$&emsp;&emsp;公式中加入了三项偏置$\mu,b_u,b_i$，其中$\mu$是训练集中所有记录的评分的全局平均数，表示网站本身对用户评分的影响；$b_u$是用户偏置项，表示用户的评分习惯中和物品没有关系的那种个人因素；$b_i$是物品偏置项，表示了物品接受的评分中和用户没有什么关系的因素。

4. 考虑邻域影响的LFM  
将ItemCF的预测算法改成如下方式：$$hat{r}_{u i}=\frac{1}{\sqrt{|N(u)|}} \sum_{j \in N(u)} w_{i j}$$其中，$w_{ij}$不再是根据ItemCF算法计算出的物品相似度矩阵，而是一个和$P$、$Q$一样的参数，可以通过优化如下的损失函数进行优化：$$C(w)=\sum_{(u, i) \in \text { Train }}\left(r_{u i}-\sum_{j \in N(u)} w_{i j} r_{u j}\right)^{2}+\lambda w_{i j}^2$$对$w$矩阵也进行分解，将参数个数降低到$2nF$个，模型如下：$$
\hat{r}_{u i}=\frac{1}{\sqrt{|N(u)|}} \sum_{j \in N(u)} x_{i}^{T} y_{j}=\frac{1}{\sqrt{|N(u)|}} x_{i}^{T} \sum_{j \in N(u)} y_{j}
$$其中， $x_i,y_j$是两个$F$维的向量。
将前面的LFM和上面的模型相加，并进行参数化简，得到最终的SVD++模型：$$
\hat{r}_{u i}=\mu+b_u+b_i+q_i^T \cdot\left(p_u+\frac{1}{\sqrt{|N(u)|}} x_i^T \sum_{j \in N(u)} y_{j}\right)$$

### 加入时间信息

1. 基于邻域的模型融合时间信息（TItemCF）   
通过如下公式预测用户在某一个时刻会给物品什么评分：$$
\hat{r}_{u i t}=\frac{\displaystyle \sum_{j \in N(u) \cap S(i, K)} f\left(w_{i j}, \Delta t\right) r_{u j}}{\displaystyle \sum_{j \in N(u) \cap S(i, K)} f\left(w_{i j}, \Delta t\right)}
$$其中，$\Delta t=t_{u i}-t_{u j}$是用户$u$对物品$i$和物品$j$评分的时间差，$w_{ij}$是物品$i$和$j$的相似度，$
f\left(w_{ij}, \Delta t\right)$是一个考虑了时间衰减后的相似度函数，可以用如下公式：$$
\begin{array}{c}
f\left(w_{i j}, \Delta t\right)=\sigma\left(\delta \cdot w_{i j} \cdot \exp \left(\displaystyle \frac{-\left|\Delta t\right|}{\beta}\right)+\gamma\right) \\
\displaystyle \sigma(x)=\frac{1}{1+\exp (-x)}
\end{array}
$$其中，$\sigma(x)$是sigmoid函数。  
可以发现，随着$\Delta t$增加，$f(w_{ij}, \Delta t)$会越来越小，也就是说用户很久之前的行为对预测用户当前评分的影响越来越小。

2. 基于矩阵分解的模型融合时间信息（TSVD）  
将用户—物品—时间三维矩阵如下分解：$$
\hat{r}_{u i t}=\mu+b_{u}+b_{i}+b_{t}+p_{u}^{T} \cdot q_{i}+x_{u}^{T} \cdot y_{t}+s_{i}^{T} z_{t}+\sum_{f} g_{u, f} h_{i, f} l_{t, f}$$其中，$b_t$表示系统整体平均分随时间变化的效应，$x_u^T \cdot y_t$表示用户平均分随时间变化的
效应，$s_i^T z_t$表示物品平均分随时间变化的效应，而$\sum_{f} g_{u, f} h_{i, f} l_{t, f}$表示用户兴趣随时间影响
的效应。  
对这个模型做如下改进以融合时间信息：$$
\begin{array}{c}
\displaystyle \hat{r}_{u it}=\mu+b_{u}(t)+b_{i}(t)+q_{i}^{T} \cdot\left(p_{u}(t)+\frac{1}{\sqrt{|N(u)|}} \sum_{j \in N(u)} y_{j}\right) \\
b_{u}(t)=b_{u}+\alpha_{u} \cdot \operatorname{dev}_{u}(t)+b_{u t}+b_{u, \text { period }(t)} \\
\operatorname{dev}_{u}(t)=\operatorname{sign}\left(t-t_{u}\right) \cdot\left|t-t_{u}\right|^{\beta} \\
b_{i}(t)=b_{i}+b_{i t}+b_{i, \text { period }(t)} \\
p_{u f}(t)=p_{u f}+p_{u t f}
\end{array}
$$其中，$t_u$是用户所有评分的平均时间。$\text{period}(t)$考虑了季节效应，可以定义为时刻$t$所在的月份。

### 模型融合

1. 模型级联融合  
假设已经有一个预测器$r^{(k)}$，对于每个用户—物品对$(u, i)$都给出预测值，那么可以在这个预测器的基础上设计下一个预测器$r^{(k+ 1)}$来最小化损失函数：$$
C=\sum_{(u, i) \in \text { Train }}\left(r_{u i}-\hat{r}_{u i}^{(k)}-\hat{r}_{u i}^{(k+1)}\right)^2
$$

In [9]:
from main.chapter8.predict_all_cascade import PredictAllCascade

metric = []
for i in range(len(UserGroups)):
    user_group = UserGroups[i]
    item_group = ItemGroups[i]
    # 模型训练
    model = PredictAllCascade(train_dataset, test_dataset, user_group, item_group)
    train_dataset = model.fit()
    test_dataset = model.predict(test_dataset)
    
    # 计算RMSE
    train_metric = RMSE(train_dataset)
    test_metric = RMSE(test_dataset)
    metric.append((user_group.__name__, item_group.__name__, train_metric, test_metric))

In [10]:
pd.DataFrame(
    data=metric,
    index=[i for i in range(len(UserGroups))],
    columns=['UserGroup', 'ItemGroup', 'TrainRMSE', 'TestRMSE']
)

,UserGroup,ItemGroup,TrainRMSE,TestRMSE
0,Cluster,Cluster,0.925377,0.999405
1,IdCluster,Cluster,0.906576,0.929917
2,Cluster,IdCluster,0.895618,0.916019
3,UserActivityCluster,Cluster,0.895582,0.916001
4,UserActivityCluster,IdCluster,0.887530,0.899072
5,Cluster,ItemPopularityCluster,0.887530,0.899071
6,IdCluster,ItemPopularityCluster,0.873239,0.884492
7,UserActivityCluster,ItemPopularityCluster,0.873238,0.884491
8,UserVoteCluster,Cluster,0.873238,0.884491
9,UserVoteCluster,IdCluster,0.865510,0.883408


2. 模型加权融合  
假设有K个不同的预测器$\{ \hat{r}^{(1)},\hat{r}^{(2)}, \cdots, \hat{r}^{(K)}\}$，最简单的融合算法是线性融合，即最终的预测器是这K个预测器的线性加权：$$
\hat{r}=\sum_{k=1}^{K} \alpha_{k} \hat{r}^{(k)}
$$在模型融合时一般采用如下方法：
 - 假设数据集已经被分为了训练集A和测试集B，那么首先需要将训练集A按照相同的分割方法分为A1和A2，其中A2的生成方法和B的生成方法一致，且大小相似。
 - 在A1上训练K个不同的预测器，在A2上作出预测。因为已知A2上的真实评分值，所以可以在A2上利用最小二乘法计算出线性融合系数$\alpha_k$。
 - 在A上训练K个不同的预测器，在B上作出预测，并且将这K个预测器在B上的预测结果，按照已经得到的线性融合系数加权融合，以得到最终的预测结果。